In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt 
%matplotlib inline
import torch.optim as optim

In [ ]:
import math

In [ ]:
np.exp(x)-1

In [ ]:
2**2

In [ ]:
# def polyfitX(x):
#     return 0.1524*(x**2) + 0.5*x + 0.409

def polyfitX(x):
    return 0.6*(math.e**x - 1)
def polyfit(x):
    return x**2
 
class Polyfit(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, input):
        return polyfitX(input)

In [ ]:
train_dir = '../input/covid-19-xray-image-dataset-with-huge-samples/COVID/train/'
test_dir = '../input/covid-19-xray-image-dataset-with-huge-samples/COVID/test/'

classes = os.listdir(train_dir)

In [ ]:
labels = ['Covid Negative', 'Covid Positive']
img_size = 150
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
            data.append([resized_arr, class_num])
    return np.array(data)

In [ ]:
import cv2
import numpy as np
img = cv2.imread("../input/covid-19-xray-image-dataset-with-huge-samples/COVID/train/Covid Negative/person100_virus_184.jpeg")
print(img.shape)

In [ ]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name):
        super(VGG, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
#         self.classifier = nn.Linear(2048, 512)
        self.classifier = nn.Linear(2048, 2)

 
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
    
   
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [ ]:
def initialize_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_uniform_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight.data, 1)
        nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class VGG_HE(nn.Module):
    def __init__(self, vgg_name):
        super(VGG_HE, self).__init__()
        self.features = self._make_layers(cfg[vgg_name])
#         self.fc1 = nn.Linear(8192, 2048)
#         self.fc2 = nn.Linear(2048, 512)
        self.classifier = nn.Linear(2048,2)
 
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
#         out = polyfitX(self.fc2(out))
        out = self.classifier(out)
        return out
    
    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                # Dùng AvgPool thay MaxPool khi dùng hàm xấp xỉ thay ReLU
                layers += [nn.AvgPool2d(kernel_size=2, stride=2)]
                
                # layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           Polyfit()]
#                             nn.ReLU(inplace=True)]
                # layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                # nn.BatchNorm2d(x),
                # nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [ ]:
train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(64),             # resize shortest side to 100 pixels
        transforms.CenterCrop(64),         # crop longest side to 100 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [ ]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(64),
    transforms.CenterCrop(64),         # crop longest side to 100 pixels at center
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [ ]:
train_ds = ImageFolder(train_dir, transform=train_transform)
print('Size of training dataset :', len(train_ds))

In [ ]:
test_ds = ImageFolder(test_dir, transform=test_transform)
print('Size of training dataset :', len(test_ds))

In [ ]:
batch_size = 64
train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2, pin_memory=True)
# val_loader = DataLoader(val_ds, batch_size*2, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size*2, num_workers=2, pin_memory=True)

In [ ]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)

In [ ]:
math.e**1 -1

In [ ]:
np.exp(1)-1

In [ ]:
print('==> Building model..')
net = VGG_HE('VGG11')
net.apply(initialize_weights)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min",patience=4)

In [ ]:
def train(epochs, dataload, model): 
    for epoch in range(0, epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        print("Epoch",epoch)
        for batch_idx , (inputs, targets) in enumerate(dataload):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print('\tLoss: '+ str(train_loss/(batch_idx+1)),
              '\tAccuracy: '+ str(100. * (correct/total)),
              "\tCorrect/total: {correct}/{total}".format(correct=correct, total=total))

In [ ]:
train(2, train_loader, net)

7 epoch dùng lr là 0.01, chính xác treen tập huấn luyện lên > 96%
Huấn luyện tiếp 7 epoch với lr = 0.001, nâng độ chính xác trên tập huấn luyện lên > 97.1%
Huấn luyện tiếp 7 epoch với lr = 0.0001, độ chính xác đạt > 97.4 %
Huấn luyện 9 epoch cuối với lr = 0.00001, độ chính xác đạt 97.65%, loss ~ 0.0725

In [ ]:
step 1 epoch 7 lr 0.01
step 2 epoch 14 lr 0.001
step 3 epoch 10 lr 0.005
step 4 epoch 10 lr 0.001

In [ ]:
def test(model, testload):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testload):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print("\nTest set: Avg. loss: {}\tAccuracy: {} \tCorrect/Total: {}/{}".format(
                test_loss/(batch_idx+1), 100. * correct/total,
                correct, total))

In [ ]:
test(net, test_loader)

Kết quả dự đoán trên tập test là 96%, loss 0.1034

In [ ]:
torch.save(net, "HE.pth")

## Not HE

In [ ]:
print('==> Building model..')
net = VGG('VGG11')
net.apply(initialize_weights)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.00001, 
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min",patience=4)

In [ ]:
train(9, train_loader, net)

In [ ]:
test(net, test_loader)

Độ chính xác trên tập test là 97.5%, loss 0.0468

In [ ]:
torch.save(net, "notHE.pth")

7 epoch dùng lr là 0.01, chính xác treen tập huấn luyện lên > 99%
Huấn luyện tiếp 7 epoch với lr = 0.001, nâng độ chính xác trên tập huấn luyện lên > 99.3%
Huấn luyện tiếp 7 epoch với lr = 0.0001, độ chính xác đạt > 97.4%
Huấn luyện 9 epoch cuối với lr = 0.00001, độ chính xác đạt 97.35%, loss ~ 0.018

step 1 epoch 7 lr 0.01
step 2 epoch 14 lr 0.001
step 3 epoch 10 lr 0.005
step 4 epoch 10 lr 0.001